In [142]:
#for aoi in list_aois[:1]:
    #shape = shapely.to_geojson(tiles.loc[aoi,"geometry"])
    #f = write_json(shape)
#    tile = tiles[tiles['random']==aoi]
    #print(type(tile))
#    tile.to_file('temp.geojson')
    

<class 'geopandas.geodataframe.GeoDataFrame'>


This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [157]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')


In [158]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely

In [159]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

In [160]:
import os

result_folder = "../aoi/WestAfricaGrid"

if not os.path.exists(result_folder):
    os.makedirs(result_folder)

for rownum, subframe in tiles.groupby("random"):
    filename = os.path.join(result_folder, f"{rownum}.geojson")
    subframe.to_file(filename, driver='GeoJSON')

len(os.listdir(result_folder))

151

In [156]:
#os.listdir(result_folder)

In [61]:
#Set index to the correct column name, in this case "random"
#tiles.set_index('random',inplace=True)
#print(tiles.head())

In [62]:
#for col in tiles.columns:
#    print(col)

In [63]:
#tiles['random_value'] = tiles.index

In [64]:
#Test a single tile
#tiles.loc[0.6855097362903453]

Use "random" column for ID to generate features to tile over

In [65]:
#List out the individual aois
#list_aois = tiles.index.unique().tolist() #tiles[tiles.columns[1]].values.tolist()
#print(list_aois)

In [66]:
#aoiTest=shapely.to_geojson(tiles.loc[0.6855097362903453,"geometry"])
#print(aoiTest)

In [161]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [162]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [41]:
#Import geojson and tempfile for creating temp files for the subsetter

#from geojson import Point, Feature, FeatureCollection, dump

#import tempfile

In [163]:
#Set up input dictionary and import backoff to display run results
#First part of the for loop uses the "random" column to search through the tiles geojson to 
#create temp files one at a time to set the AOI
#################################

for aoi in os.listdir(result_folder):
    if aoi.endswith("geojson"):
        inputs = dict(
           aoi=f"https://maap-ops-workspace.s3.amazonaws.com/shared-buckets/abarenblitt/aoi/WestAfricaGrid/{aoi}",
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 10,
           temporal="-",
           output="-"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset_ubuntu",
            version="0.6.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )

        job_id = result["job_id"]
        print(job_id)
        
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

527a84de-3e8a-46ca-ae0d-d853fcb183f2
4061aad8-87cb-4d6b-bec8-91985f183f80
9da41252-b1b0-43f0-98a4-503ab7b079dc
ee5ad41b-09ff-4d61-8e25-b37695214689
563c1913-459c-43cb-bd26-a962b6143cbb
e0b52abc-4cd9-44a8-8915-e83a124029e1
a539c01e-327b-48d6-b39f-c701de1f2784
441f495b-beba-4a74-98a5-1fdd2284b286
27bc5700-3c03-4886-b505-400c86bcbec1
c61127bd-060a-4e39-ada9-9bba78a695fb
3c44d23a-6905-4362-8594-9288d614ddc7
2a641db0-518d-416d-9940-c5763ff108df
551cc0d6-f007-47a1-a791-44605893a8d1
36329b60-864c-407b-b415-e37c684d52e4
ca6daa00-1f2b-423f-9080-415cf4741625
387d2ab8-448f-4ea0-b3e4-5f7f578b7ba5
f2dcc96e-666d-4dea-9ae1-48f6e4c805ac
359c5242-b69c-499d-b075-7c06147f1601
91f51746-6b2a-4e05-94c0-f4e3249f4fec
bc8d70df-013e-415f-a4ea-e34772bc28fd
fd6e1f15-d25a-4ff1-89f8-60d2ea485e4e
f52bc5b3-c34b-4bd6-9f80-c7cf3962a594
725530cb-39db-4a92-95da-75f6c5b26e00
4d5722c2-e494-4283-a57d-3652b83443c9
0443a8a5-615c-4e37-8b81-1a26ae3ada22
92feb670-ba65-4313-849b-6dc6cedd320f
b9b87482-4d95-42d9-8bac-883901ee2b3a
f

INFO:backoff:Backing off wait_for_job(...) for 33.9s (Accepted)


ff3c1e89-aaf6-4bb1-99af-03ad5263e3f1
fcf5e7d3-3343-43db-9fc5-71406065969f


INFO:backoff:Backing off wait_for_job(...) for 114.4s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 69.5s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 21.6s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 62.3s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 31.9s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 88.6s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 63.1s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 84.3s (Running)


In [164]:
job_id = "fcf5e7d3-3343-43db-9fc5-71406065969f"

job_status

'Failed'

In [165]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>fcf5e7d3-3343-43db-9fc5-71406065969f</wps:JobID><wps:Output id="traceback"><wps:Data>Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/utils.py", line 139, in download_file\n    osaka.main.get(url, cache_dir, params=params)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 86, in get\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 128, in transfer\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", line 128, in transfer\n    raise err\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", line 106, in transfer\n    metrics = self.transfer_uri(source, shandle, dest, dhandle)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/transfer.py", li